In [1]:
from tqdm.notebook import tqdm
import csv

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [8]:
device = 'cuda'
model_name = 'EleutherAI/gpt-j-6B'
model_precision = "float16"
target_token_idx = 11
max_length = 200
gen_length = 20
input_fn = '../out/memorization/head_val_extract.csv'
output_fn = '../out/memorization/head_val_scored.csv'

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name, truncation_side='left')
if model_precision == "float16":
    model = AutoModelForCausalLM.from_pretrained(model_name, revision="float16", torch_dtype=torch.float16,
                                                 return_dict=True).to(device)
else:
    model = AutoModelForCausalLM.from_pretrained(model_name, return_dict=True).to(device)

In [9]:
in_data = list(csv.reader(open(input_fn, 'rt')))
header = in_data[0]
in_data = in_data[1:]

In [10]:
out_fh = open(output_fn, 'wt')
out = csv.writer(out_fh)

# write headers
out.writerow(["line", "prefix", "suffix", "generated"])

30

In [ ]:
for i, line in tqdm(enumerate(in_data), total=len(in_data)):
    line_idx, prefix, suffix, suffix_len = line

    input_ids = tokenizer.encode(prefix, \
                                 return_tensors='pt', \
                                 max_length=max_length, \
                                 truncation=True, \
                                 padding=False).to(device)
    # i checked, it is left truncate

    with torch.no_grad():
        model.eval()
        outputs = model.greedy_search(input_ids, pad_token_id=-1, 
                                      max_length=len(input_ids[0])+gen_length)

    outputs = outputs.tolist()[0]
    outputs = outputs[-gen_length:]
    output_str = tokenizer.decode(outputs)

    out.writerow([line_idx, prefix, suffix, output_str])

  0%|          | 0/20000 [00:00<?, ?it/s]

In [ ]:
out_fh.close()